In [ ]:
%run "0.0 Data preparation.ipynb"

In [ ]:
positiveQuestions = indexedQuestions[indexedQuestions.index % 2 == 1]
negativeQuestions = indexedQuestions[0:11]
negativeQuestions = negativeQuestions[negativeQuestions.index % 2 == 0]

In [ ]:
print("Best scores:")
for question in indexedQuestions[0:11]:
    bestGame = ""
    bestScore = 0
    for game in games:
        questionScore = getSUSScore(question, game, data=raw20190603, printResult=False)
        if questionScore > bestScore:
            bestGame = game
            bestScore = questionScore
    print("'" + question + "': " + bestGame + " " + '{:1.1f}%'.format(bestScore))

In [ ]:
print("Worst scores:")
for question in indexedQuestions[0:11]:
    worstGame = ""
    worstScore = 100
    for game in games:
        questionScore = getSUSScore(question, game, data=raw20190603, printResult=False)
        if questionScore < worstScore:
            worstGame = game
            worstScore = questionScore
    print("'" + question + "': " + worstGame + " " + '{:1.1f}%'.format(worstScore))

In [ ]:
def getSUSScore(question, game, data=raw20190603, printResult=True):
    _score = 0
    _maxScore = 0
    _gameQuestionData = data.loc[data[gameQuestion]==game, question]

    #globalScore.sum(axis=0)
    if question in positiveQuestions.values:
#        print(question)
        for _answer in _gameQuestionData:
            if pd.notna(_answer):
                _score += (5-int(_answer))
                _maxScore += 4
    else:
#        print(question)
        for _answer in _gameQuestionData:
            if pd.notna(_answer):
                _score += (int(_answer)-1)
                _maxScore += 4
    _percentageScoreResult = _score*100/_maxScore
    if printResult:
        print("'" + question + "': " + game + " " + '{:1.1f}%'.format(_percentageScoreResult))
    return _percentageScoreResult
    

In [ ]:
allSUSScores = pd.DataFrame(index=indexedQuestions.values[0:11], columns=games)
for game in games:
    for question in indexedQuestions.values[0:11]:
        allSUSScores.loc[question, game] = getSUSScore(question, game, printResult=False)

allSUSScores = allSUSScores.astype(float)

In [ ]:
f = plt.figure()
ax = f.add_subplot(111)
h = sns.heatmap(allSUSScores,
            ax=ax,
            cmap=plt.cm.jet,
            vmin=50,
            vmax=100,
            square=True,
            cbar_kws = dict(use_gridspec=False,location="left")
           )
ax.yaxis.tick_right()
plt.yticks(rotation=0);
#h.set_xticklabels(ax.get_xticklabels(), rotation=30)

In [ ]:
pd.concat((allSUSScores.idxmin(), allSUSScores.min()), 1)

In [ ]:
pd.concat((allSUSScores.idxmax(), allSUSScores.max()), 1)

## how to display agreement and variance of answers?

In [ ]:
raw20190603

In [ ]:
1 or 2

In [ ]:
def getVarianceOnQG(question, game, data=raw20190603, printResult=True):
#    print("Q='"+question+"'; G="+game)
    data=raw20190603
    gameQuestionData = data.loc[data[gameQuestion]==game, question]
    gameQuestionData = [int(v) for v in gameQuestionData if pd.notna(v)]
    return np.var(gameQuestionData)

In [ ]:
allVars = pd.DataFrame(index=indexedQuestions.values[0:11], columns=games)
for game in games:
    for question in indexedQuestions.values[0:11]:
        allVars.loc[question, game] = getVarianceOnQG(question, game, printResult=False)

allVars = allVars.astype(float)

In [ ]:
f = plt.figure()
ax = f.add_subplot(111)
h = sns.heatmap(allVars,
            ax=ax,
            cmap=plt.cm.jet,
            vmin=0.4,
            vmax=2.2,
            square=True,
            cbar_kws = dict(use_gridspec=False,location="left")
           )
ax.yaxis.tick_right()
plt.yticks(rotation=0);
#h.set_xticklabels(ax.get_xticklabels(), rotation=30)

In [ ]:
pd.concat((allVars.idxmin(), allVars.min()), 1)

In [ ]:
pd.concat((allVars.idxmax(), allVars.max()), 1)